<a href="https://colab.research.google.com/github/hannah-rae/gee_tutorials/blob/main/SES230_GEE_lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SES 230 Lab
# Satellite Data Analysis in Google Earth Engine

In the [lecture](https://colab.research.google.com/drive/17IPwrpeK4sAnGy1MGiSVmdT3dc6DXMmM?authuser=1#scrollTo=2vUR2ZLiFh9l) tutorial, we learned how to use GEE to analyze satellite data in the cloud using Landsat, Sentinel-2, band indices, and K-means clustering. 

In the lab, we'll explore this further with your own ROIs.

## Set up your environment

Install the Google Earth Engine API

In [ ]:
!pip install earthengine-api

Authenticate your Google Earth Engine account

In [ ]:
!earthengine authenticate

Import the Earth Engine API and initialize it.

In [ ]:
import ee
ee.Initialize()

Install `geemap`, a python library that provides useful functions for the GEE Python API (https://github.com/giswqs/geemap)

In [ ]:
!pip install geemap

Import `geemap`. We import the `eefolium` version because the default version uses `ipyleaflet`, which is not supported in Colab yet.

In [ ]:
import geemap.eefolium as geemap

Mount your Google Drive account as your filesystem for this Colab notebook. This is needed if you want to export data from Google Earth Engine, or you want to open any files from your Google Drive.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Define a region of interest (ROI)

http://bboxfinder.com can be used to draw a bounding box and get the coordinates for that bounding box anywhere in the world.

In the lecture, we used this one around Tempe, AZ:
http://bboxfinder.com/#33.406947,-111.971585,33.473126,-111.907899

Use bboxfinder to draw your own bounding box anywhere in the world and enter the min/max longitude/latitude below:

In [ ]:
# Enter the min lon, min lat, max lon, max lat
xmin,ymin,xmax,ymax = # your code here

Create an `ee.Geometry.Rectangle` object defined by those coordinates.

In [ ]:
bbox = ee.Geometry.Rectangle([xmin,ymin,xmax,ymax])

Then we add your bounding box to the map and give it a name.

In [ ]:
Map = geemap.Map() # Instantiate a new map
Map.addLayer(bbox, name='your name here') # Add the bbox as a layer on the map
Map.centerObject(bbox, zoom=10) # Center the Map on the bbox object with zoom level 10

Map.addLayerControl() 
Map

## Load satellite data sets

Load the Sentinel-2 image collection and filter it by your ROI. For the time range, use the beginning of 2020 through November 1, 2020.

In [ ]:
s2 = ee.ImageCollection('COPERNICUS/S2_SR').filterBounds(# your code here).filterDate([#your code here])

Display the image collection.

In [ ]:
visParams = {
  'bands': ['B4', 'B3', 'B2'],
  'min': 0,
  'max': 3000,
  'gamma': 1.4,
}

In [ ]:
Map = geemap.Map() # Instantiate a new map
Map.addLayer(bbox, name='your name here') # Add the bbox as a layer on the map
Map.centerObject(bbox, zoom=10) # Center the Map on the bbox object with zoom level 10

Map.addLayer(s2, visParams, name='Sentinel-2') # Add the Sentinel-2 collection for your ROI

Map.addLayerControl() 
Map

Create a minimum value composite image from the collection and clip to to your ROI.

In [ ]:
s2 = s2.min().clip(# your code here)

In [ ]:
Map = geemap.Map() # Instantiate a new map
Map.addLayer(bbox, name='your name here') # Add the bbox as a layer on the map
Map.centerObject(bbox, zoom=10) # Center the Map on the bbox object with zoom level 10

Map.addLayer(s2, visParams, name='Sentinel-2') # Add the Sentinel-2 collection for your ROI

Map.addLayerControl() 
Map

## Compute band indices

Compute normalized difference vegetation index, or NDVI (NIR-RED/NIR+RED)

In [ ]:
ndvi = # your code here

In [ ]:
ndvi_vis = {
    'min': -1,
    'max':1,
    'palette': ['3498DB', 'FFFFFF', '008000']
}

In [ ]:
Map = geemap.Map() # Instantiate a new map
Map.addLayer(bbox, name='your name here') # Add the bbox as a layer on the map
Map.centerObject(bbox, zoom=10) # Center the Map on the bbox object with zoom level 10

Map.addLayer(s2, visParams, name='Sentinel-2') # Add the Sentinel-2 collection for your ROI
Map.addLayer(ndvi, ndvi_vis, name='NDVI') # Add the NDVI image

Map.addLayerControl() 
Map

There are many other indices that scientists use in addition to NDVI. One popular index is the normalized difference water index (NDWI). 

NDWI is computed as: (GREEN-NIR)/(GREEN+NIR)

In [ ]:
ndwi = # your code here

The colors we chose for the NDVI color map were based on what the values of NDVI might show (e.g., low values = water = blue, high values = vegetation = green). 

For NDWI, let's go from brown to white to blue.

In [ ]:
ndwi_vis = {
    'min': -1,
    'max':1,
    'palette': ['964B00', 'FFFFFF', '3498DB']
}

In [ ]:
Map = geemap.Map() # Instantiate a new map
Map.addLayer(bbox, name='your name here') # Add the bbox as a layer on the map
Map.centerObject(bbox, zoom=10) # Center the Map on the bbox object with zoom level 10

Map.addLayer(s2, visParams, name='Sentinel-2') # Add the Sentinel-2 collection for your ROI
Map.addLayer(ndvi, ndvi_vis, name='NDVI') # Add the NDVI image
Map.addLayer(ndwi, ndwi_vis, name='NDWI') # Add the NDWI image

Map.addLayerControl() 
Map

## Cluster analysis

In the lecture, we clustered NDVI values to reveal land cover patterns. Now, let's cluster NDWI values.

Create the training data set then train the clusterer.

In [ ]:
# Make the training dataset.
training = ndwi.sample(**{
  'region': bbox, # restrict the sampling to our bounding box
  'scale': 10, # sample within 30m/pixel cells, i.e. the data set resolution
  'numPixels': 1000 # number of samples to draw
})

In [ ]:
clusterer = ee.Clusterer.wekaKMeans(# your code here).train(training)

Cluster (predict on) the data set using the trained clusterer model.

In [ ]:
result = ndwi.cluster(clusterer)

Visualize the clusters.

In [ ]:
Map = geemap.Map() # Instantiate a new map
Map.addLayer(bbox, name='your name here') # Add the bbox as a layer on the map
Map.centerObject(bbox, zoom=10) # Center the Map on the bbox object with zoom level 10

Map.addLayer(s2, visParams, name='Sentinel-2') # Add the Sentinel-2 collection for your ROI
Map.addLayer(ndwi, ndwi_vis, name='NDWI') # Add the NDWI image
Map.addLayer(result.randomVisualizer(), name='Clusters') # Add the cluster results

Map.addLayerControl() 
Map

## Save your results

Hooray! You've made it to the end. Save your NDWI image and cluster results as images.

In [ ]:
task = ee.batch.Export.image.toDrive(image=ndwi, 
                                           description='SES 230 NDWI', 
                                           scale=10, 
                                           region=bbox)

In [ ]:
task.start()

In [ ]:
task2 = ee.batch.Export.image.toDrive(image=result, 
                                           description='SES 230 Clusters', 
                                           scale=10, 
                                           region=bbox)

In [ ]:
task2.start()

In [ ]:
task.status()

In [ ]:
task2.status()

## Bonus: load a new satellite data set

Explore the GEE catalog and display a different data set within your ROI. 

https://developers.google.com/earth-engine/datasets

Some ideas are:
*   Thermal IR: https://developers.google.com/earth-engine/datasets/catalog/ASTER_AST_L1T_003
*   Nighttime lights: https://developers.google.com/earth-engine/datasets/catalog/NOAA_DMSP-OLS_NIGHTTIME_LIGHTS?hl=en
* Precipitation: https://developers.google.com/earth-engine/datasets/catalog/NASA_GPM_L3_IMERG_V06



In [ ]:
# your code here